In [1]:
pip install lightgbm

In [2]:
pip install shap

In [23]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.0 MB/s eta 0:00:00


In [24]:
import argparse
import os
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve,
    accuracy_score, f1_score, precision_score, recall_score
)
from sklearn.inspection import permutation_importance

from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler

In [25]:
try:
    import shap
    _HAS_SHAP = True
except Exception:
    _HAS_SHAP = False

In [26]:
CSV_PATH = "cleaned_exoplanets.csv"
TEST_SIZE = 0.2
RANDOM_STATE = 42
SAVE_MODEL_PATH = "exo_lgbm_binary_enhanced.joblib"
OPTIMIZE_HYPERPARAMS = True  # Set to True to run optimization
N_TRIALS = 20  # Number of optimization trials
OUTPUT_DIR = "model_outputs"

In [27]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_data(path):
    df = pd.read_csv(path)
    return df

In [45]:
def engineer_features(X):
    """Enhanced feature engineering with more sophisticated features"""

    # Original features
    if 'depth_ppm' in X.columns:
        X['depth_ratio'] = X['depth_ppm'] / 1e6
    if 'st_rad_re' in X.columns and 'radius_re' in X.columns:
        X['st_rad_rj'] = X['st_rad_re'] / 11.2
        X['ror_check'] = X['radius_re'] / (X['st_rad_re'] + 1e-9)
    if 'ror' in X.columns and 'ror_check' in X.columns:
        X['ror_diff'] = (X['ror'] - X['ror_check']).abs()
    if 'duration_hours' in X.columns and 'period_days' in X.columns:
        X['duration_fraction'] = X['duration_hours'] / (X['period_days'] * 24 + 1e-9)
    if 'depth_ppm' in X.columns and 'duration_hours' in X.columns:
        X['depth_per_hour'] = X['depth_ppm'] / (X['duration_hours'] + 1e-9)
    if 'teq_k' in X.columns and 'st_teff_k' in X.columns:
        X['teq_ratio'] = X['teq_k'] / (X['st_teff_k'] + 1e-9)
    if 'insolation_se' in X.columns and 'st_rad_re' in X.columns:
        X['insolation_scaled'] = X['insolation_se'] / (X['st_rad_re']**2 + 1e-6)
    if 'period_days' in X.columns:
        X['log_period'] = np.log1p(X['period_days'])

    # NEW: Advanced features
    # Transit depth consistency
    if 'radius_re' in X.columns and 'st_rad_re' in X.columns and 'depth_ppm' in X.columns:
        X['expected_depth'] = (X['radius_re'] / (X['st_rad_re'] + 1e-9))**2 * 1e6
        X['depth_anomaly'] = (X['depth_ppm'] - X['expected_depth']).abs() / (X['expected_depth'] + 1e-3)

    # Orbital characteristics
    if 'semi_major_axis_au' in X.columns:
        X['log_semi_major'] = np.log1p(X['semi_major_axis_au'])
    if 'semi_major_axis_au' in X.columns and 'period_days' in X.columns:
        X['orbital_velocity'] = 2 * np.pi * X['semi_major_axis_au'] / (X['period_days'] / 365.25 + 1e-9)

    # Stellar context
    if 'radius_re' in X.columns and 'st_rad_re' in X.columns:
        X['planet_star_radius_ratio'] = X['radius_re'] / (X['st_rad_re'] + 1e-9)
    if 'st_mass_ms' in X.columns and 'st_rad_re' in X.columns:
        X['stellar_density_proxy'] = X['st_mass_ms'] / (X['st_rad_re']**3 + 1e-9)

    # Temperature ratios
    if 'teq_k' in X.columns and 'st_teff_k' in X.columns:
        X['temp_contrast'] = X['teq_k'] / (X['st_teff_k'] + 1e-9)
    if 'teq_k' in X.columns:
        X['log_teq'] = np.log1p(X['teq_k'])

    # Transit geometry
    if 'inclination_deg' in X.columns:
        X['impact_param_proxy'] = X['inclination_deg'] / 90.0
    if 'st_rad_rj' in X.columns and 'semi_major_axis_au' in X.columns:
        X['transit_prob'] = (X['st_rad_rj'] + 1e-9) / (X['semi_major_axis_au'] * 215.032 + 1e-9)  # 215.032 = AU to R_sun

    # Signal strength indicators
    if 'depth_ppm' in X.columns and 'duration_hours' in X.columns:
        X['signal_strength'] = X['depth_ppm'] * np.sqrt(X['duration_hours'])
        X['snr_proxy'] = X['depth_ppm'] / (X['duration_hours'] + 1e-9)

    # Interaction features
    if 'log_period' in X.columns and 'depth_ratio' in X.columns:
        X['period_depth_interaction'] = X['log_period'] * X['depth_ratio']
    if 'radius_re' in X.columns and 'insolation_se' in X.columns:
        X['radius_insolation'] = X['radius_re'] * np.log1p(X['insolation_se'])

    return X

In [46]:
def prepare_data(df):
    """Prepare data with enhanced feature engineering"""
    # Keep only CONFIRMED and FALSE POSITIVE rows
    df = df[df["disposition"].astype(str).str.upper().isin(["CONFIRMED", "FALSE POSITIVE"])].copy()

    # Encode target: 1 = CONFIRMED, 0 = FALSE POSITIVE
    y = df["disposition"].astype(str).str.upper().map({
        "FALSE POSITIVE": 0,
        "CONFIRMED": 1
    }).astype(int)

    drop_cols = ["disposition", "src_rowid", "unified_id", "id_bundle", "host_name", 'mission', 't0_bjd']
    feature_cols = [c for c in df.columns if c not in drop_cols]
    X = df[feature_cols].copy()

    # Apply enhanced feature engineering
    X = engineer_features(X)

    # Get numeric columns (after feature engineering)
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

    return X, y, num_cols


In [47]:
def objective(trial, X_train, y_train):
    """Optuna objective function for hyperparameter optimization"""
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

    clf = LGBMClassifier(
        **param,
        objective="binary",
        class_weight="balanced",
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )

    # Use stratified k-fold cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='f1', n_jobs=-1)

    return scores.mean()


In [48]:
def optimize_hyperparameters(X_train, y_train, n_trials=100):
    """Run hyperparameter optimization"""
    print(f"\n=== Running Hyperparameter Optimization ({n_trials} trials) ===")

    study = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=RANDOM_STATE)
    )

    study.optimize(
        lambda trial: objective(trial, X_train, y_train),
        n_trials=n_trials,
        show_progress_bar=True
    )

    print(f"\nBest F1 Score: {study.best_value:.4f}")
    print("\nBest Hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    return study.best_params


In [49]:
def build_pipeline(num_cols, hyperparams=None):
    """Build pipeline with optional optimized hyperparameters"""
    pre = ColumnTransformer([
        ("num", "passthrough", num_cols),
    ])

    if hyperparams is None:
        # Default parameters (improved from original)
        hyperparams = {
            'n_estimators': 800,
            'learning_rate': 0.03,
            'max_depth': 8,
            'num_leaves': 63,
            'subsample': 0.9,
            'colsample_bytree': 0.9,
            'min_child_samples': 20,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
        }

    clf = LGBMClassifier(
        **hyperparams,
        objective="binary",
        class_weight="balanced",
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    pipe = Pipeline([
        ("prep", pre),
        ("clf", clf)
    ])
    return pipe


In [50]:
def plot_confusion_matrix(y_test, preds, save_path):
    """Plot enhanced confusion matrix"""
    cm = confusion_matrix(y_test, preds)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['FALSE POSITIVE', 'CONFIRMED'],
                yticklabels=['FALSE POSITIVE', 'CONFIRMED'],
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)

    # Add percentages
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j+0.5, i+0.7, f'({cm_norm[i, j]*100:.1f}%)',
                    ha='center', va='center', fontsize=10, color='gray')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved confusion matrix to {save_path}")


In [51]:
def plot_roc_curve(y_test, y_proba, save_path):
    """Plot ROC curve"""
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved ROC curve to {save_path}")


In [52]:
def plot_precision_recall_curve(y_test, y_proba, save_path):
    """Plot Precision-Recall curve"""
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.4f})')
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title('Precision-Recall Curve', fontsize=14, fontweight='bold')
    plt.legend(loc="lower left")
    plt.grid(alpha=0.3)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved PR curve to {save_path}")


In [53]:
def plot_feature_importance(model, num_cols, save_path, top_n=20):
    """Plot feature importance from LightGBM"""
    clf = model.named_steps["clf"]

    feature_imp = pd.DataFrame({
        'feature': num_cols,
        'importance': clf.feature_importances_
    }).sort_values('importance', ascending=False).head(top_n)

    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_imp, y='feature', x='importance', palette='viridis')
    plt.title(f'Top {top_n} Feature Importances (LightGBM)', fontsize=14, fontweight='bold')
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved feature importance plot to {save_path}")


In [54]:
def plot_permutation_importance(imp_df, save_path, top_n=20):
    """Plot permutation importance"""
    imp_top = imp_df.head(top_n).sort_values('mean_importance')

    plt.figure(figsize=(10, 8))
    plt.barh(range(len(imp_top)), imp_top['mean_importance'],
             xerr=imp_top['std'], color='coral', alpha=0.8)
    plt.yticks(range(len(imp_top)), imp_top['feature'])
    plt.xlabel('Mean Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title(f'Top {top_n} Permutation Importances', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved permutation importance plot to {save_path}")


In [55]:
def plot_shap_summary(model, X_test, save_path, max_display=20):
    """Plot SHAP summary if available"""
    if not _HAS_SHAP:
        print("SHAP not available, skipping SHAP plots")
        return

    try:
        # Transform test data
        X_trans = model.named_steps["prep"].transform(X_test)
        clf = model.named_steps["clf"]

        # Create SHAP explainer
        explainer = shap.TreeExplainer(clf)
        shap_values = explainer.shap_values(X_trans)

        # Handle binary classification output
        if isinstance(shap_values, list):
            shap_values = shap_values[1]  # Use positive class

        # Create summary plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_trans,
                         feature_names=X_test.columns.tolist(),
                         max_display=max_display, show=False)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved SHAP summary plot to {save_path}")
    except Exception as e:
        print(f"Error creating SHAP plot: {e}")


In [56]:
def evaluate(model, X_test, y_test):
    """Comprehensive model evaluation with visualizations"""
    preds = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Text metrics
    print("\n" + "="*60)
    print("CLASSIFICATION REPORT")
    print("="*60)
    print(classification_report(y_test, preds,
                                target_names=["FALSE POSITIVE", "CONFIRMED"],
                                digits=4))

    # Summary metrics
    print("\n" + "="*60)
    print("SUMMARY METRICS")
    print("="*60)
    print(f"Accuracy:  {accuracy_score(y_test, preds):.4f}")
    print(f"Precision: {precision_score(y_test, preds):.4f}")
    print(f"Recall:    {recall_score(y_test, preds):.4f}")
    print(f"F1-Score:  {f1_score(y_test, preds):.4f}")

    # ROC AUC
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    print(f"ROC AUC:   {roc_auc:.4f}")

    # Confusion Matrix (text)
    print("\n" + "="*60)
    print("CONFUSION MATRIX")
    print("="*60)
    cm_df = pd.DataFrame(
        confusion_matrix(y_test, preds),
        index=["TRUE: FALSE POSITIVE", "TRUE: CONFIRMED"],
        columns=["PRED: FALSE POSITIVE", "PRED: CONFIRMED"]
    )
    print(cm_df)

    # Generate all plots
    print("\n" + "="*60)
    print("GENERATING VISUALIZATIONS")
    print("="*60)

    plot_confusion_matrix(y_test, preds,
                         os.path.join(OUTPUT_DIR, 'confusion_matrix.png'))
    plot_roc_curve(y_test, y_proba,
                  os.path.join(OUTPUT_DIR, 'roc_curve.png'))
    plot_precision_recall_curve(y_test, y_proba,
                               os.path.join(OUTPUT_DIR, 'precision_recall_curve.png'))

    return preds, y_proba


In [57]:
def _feature_names_from_preprocessor(prep):
    """Build feature names from ColumnTransformer"""
    names = []
    for name, trans, cols in prep.transformers_:
        if name == "remainder":
            continue
        if trans == "drop":
            continue
        if hasattr(trans, "get_feature_names_out"):
            try:
                base = cols if isinstance(cols, (list, tuple, np.ndarray)) else [cols]
                names.extend(list(trans.get_feature_names_out(base)))
            except Exception:
                names.extend(list(trans.get_feature_names_out()))
        else:
            names.extend(list(cols) if isinstance(cols, (list, tuple, np.ndarray)) else [cols])
    return names


In [58]:
def explain_model(model, X_test, y_test, num_cols):
    """Generate all explanation plots"""
    print("\n" + "="*60)
    print("MODEL EXPLANATIONS")
    print("="*60)

    # Feature importance from model
    plot_feature_importance(model, num_cols,
                          os.path.join(OUTPUT_DIR, 'feature_importance.png'))

    # Permutation importance
    print("\nCalculating permutation importance...")
    X_trans = model.named_steps["prep"].transform(X_test)
    r = permutation_importance(
        model.named_steps["clf"],
        X_trans,
        y_test,
        n_repeats=10,
        random_state=RANDOM_STATE,
    )

    feat_names = _feature_names_from_preprocessor(model.named_steps["prep"])
    imp_df = pd.DataFrame({
        "feature": feat_names,
        "mean_importance": r.importances_mean,
        "std": r.importances_std
    }).sort_values("mean_importance", ascending=False)

    print("\nTop 15 Features (Permutation Importance):")
    print(imp_df.head(15).to_string(index=False))

    plot_permutation_importance(imp_df,
                               os.path.join(OUTPUT_DIR, 'permutation_importance.png'))

    # SHAP values
    if _HAS_SHAP:
        print("\nGenerating SHAP explanations...")
        plot_shap_summary(model, X_test,
                         os.path.join(OUTPUT_DIR, 'shap_summary.png'))


In [59]:
def main():
    print("="*60)
    print("EXOPLANET DETECTION MODEL")
    print("="*60)

    # Load and prepare data
    print("\nLoading data")
    df = load_data(CSV_PATH)
    X, y, num_cols = prepare_data(df)

    print(f"Dataset shape: {X.shape}")
    print(f"Number of features: {len(num_cols)}")
    print(f"Class distribution:\n{pd.Series(y).value_counts()}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

    # Optimize hyperparameters (optional)
    best_params = None
    if OPTIMIZE_HYPERPARAMS:
        best_params = optimize_hyperparameters(X_train, y_train, n_trials=N_TRIALS)

    # Build and train model
    print("\nTraining model...")
    model = build_pipeline(num_cols, best_params)
    model.fit(X_train, y_train)

    # Evaluate with visualizations
    preds, y_proba = evaluate(model, X_test, y_test)

    # Generate explanations
    explain_model(model, X_test, y_test, num_cols)

    # Save model
    if SAVE_MODEL_PATH:
        import joblib
        joblib.dump(model, SAVE_MODEL_PATH)
        print(f"\n[INFO] Model saved to {SAVE_MODEL_PATH}")

    print("\n" + "="*60)
    print(f"All visualizations saved to '{OUTPUT_DIR}/' directory")
    print("="*60)


In [60]:

if __name__ == "__main__":
    main()

[I 2025-10-05 17:20:45,133] A new study created in memory with name: no-name-2379afad-5ee6-49c4-b617-851e4c252681


EXOPLANET DETECTION MODEL

Loading data
Dataset shape: (10147, 28)
Number of features: 28
Class distribution:
disposition
0    6134
1    4013
Name: count, dtype: int64

=== Running Hyperparameter Optimization (20 trials) ===


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-10-05 17:21:46,461] Trial 0 finished with value: 0.8709995760338305 and parameters: {'n_estimators': 500, 'learning_rate': 0.08927180304353628, 'max_depth': 10, 'num_leaves': 98, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'min_child_samples': 10, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598}. Best is trial 0 with value: 0.8709995760338305.
[I 2025-10-05 17:23:43,248] Trial 1 finished with value: 0.8726794294844862 and parameters: {'n_estimators': 767, 'learning_rate': 0.010485387725194618, 'max_depth': 12, 'num_leaves': 129, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.6727299868828402, 'min_child_samples': 22, 'reg_alpha': 5.472429642032198e-06, 'reg_lambda': 0.00052821153945323}. Best is trial 1 with value: 0.8726794294844862.
[I 2025-10-05 17:24:24,735] Trial 2 finished with value: 0.8699265668636571 and parameters: {'n_estimators': 545, 'learning_rate': 0.019553708662745254, 'max_depth': 9, 'num_leaves': 38, 'sub